<a href="https://colab.research.google.com/github/gpufit/Comet/blob/master/COMET.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# COMET 
Cost-function Optimized Maximal overlap drift EsTimation

COMET is a software package designed to correct drift in single molecule localization (SMLM)
datasets with a high spatial and temporal resolution. 

Check out the [Github repository](https://github.com/gpufit/Comet) for more detailed information.




# Usage of this notebook

## Step 0: Set up the notebook
Before you run the notebook, please ensure that you are logged into your Google account. Go to file-> save a copy in Drive. Now that you have your own copy of this notebook, go to edit -> notebook-settings and activate the gpu hardware-acceleration. 





## Step 1: Importing a dataset
As a first step, you need to upload and import the SMLM dataset, that you want to drift-correct. 
With the following lines of code you can import a file from your local computer, assuming it is saved in the Thunderstorm format. 

In [ ]:
from google.colab import files
file = files.upload()
import numpy as np 
import pandas as pd 
for key in file.keys():
  filename = key
import io
data = pd.read_csv(io.BytesIO(file[filename]))

localizations = np.zeros((len(data['frame']), 4))
localizations[:, 0] = np.asarray(data['x [nm]'])
localizations[:, 1] = np.asarray(data['y [nm]'])
localizations[:, 2] = np.asarray(data['z [nm]'])
localizations[:, 3] = np.asarray(data['frame'])
frames = np.unique(localizations[:, -1])
n_frames = len(frames)
print(f"{filename} import successful, {len(localizations[:, 0])} localizations, {n_frames} frames")

Saving image_localizations_no_drift_corr_cropped.csv to image_localizations_no_drift_corr_cropped.csv


... alternatively you can directly import them from your google drive using the follwoing code and replacing "test_dataset.csv" with your filename

In [1]:
from google.colab import drive
drive.mount('/content/gdrive')
path_to_drive_files = "gdrive/MyDrive/"
filename = "test_dataset.csv"
import numpy as np 
import pandas as pd 

data = pd.read_csv(f"{path_to_drive_files}{filename}")

localizations = np.zeros((len(data['frame']), 4))
localizations[:, 0] = np.asarray(data['x [nm]'])
localizations[:, 1] = np.asarray(data['y [nm]'])
localizations[:, 2] = np.asarray(data['z [nm]'])
localizations[:, 3] = np.asarray(data['frame'])
frames = np.unique(localizations[:, -1])
n_frames = len(frames)
print(f"{filename} import successful, {len(localizations[:, 0])} localizations, {n_frames} frames")

Mounted at /content/gdrive
test_dataset.csv import successful, 300000 localizations, 14965 frames



#### how to import other formats?
If you use a different data format, modify the lines of code so that you end up with a numpy array called localizations that has the following dimensions:

localizations.shape = (number_of_localizations, dataset_dimension+1), where localizations[:, 0] are the x-coordinates etc. and localizations[:, -1] are the corresponding frames.

In [2]:
from google.colab import files

file = files.upload()

"""
Your import code here
"""
assert (len(localizations[0,:])==3 or len(localizations[0,:])==4)
frames = np.unique(localizations[:, -1])
n_frames = len(frames)

## Step 2: Segment dataset
In the next step, the dataset is going to be segmented. Currently we provide 3 segmentation methods, but of cause feel free to try out your own by modifying the following lines of code.

1) **segment by number of time windows**: 

this divides the dataset in equal parts containing a fixed 
number of localizations, you can specify the number of parts. 

2) **segments by number of locs per window**

similar to option 1)
but here you have to specify the number of localizations per 
window and the resulting number of segments is calculated. 

3) **segment by number of frames per window**:

this will use the information provided by the dataset and split the dataset in unequal parts, where each part contains all the localizations witin a number of frames that you specify.

Run the next few lines to first define the segmentation methods and the code after to apply one of the segmentation methods to the earlier imported localizations. The default here is method 1 with 200 segments, feel free to to play around with the segmentation methods and try to find the optimal parameters. 

-> Increasing the number of segments will increase the time resolution of the drift estimation, but makes the estimation less robust or in extreme cases can lead to wrong estimates. 


In [3]:
segmentation_method = 'segment by number of time windows' #@param ["'segment by number of time windows'", "'segments by number of locs per window'", "'segment by number of frames per window'"] {type:"raw", allow-input: true}
segmentation_parameter = 300 #@param{type:"integer"}

def segment_by_num_windows(locs_nm, n_windows, return_n_segments=False):
    n_windows = int(n_windows)
    n_locs = len(locs_nm[:, 0])
    n_locs_per_window = int(np.ceil(n_locs/n_windows))
    if return_n_segments:
        return segment_by_num_locs_per_window(locs_nm, n_locs_per_window), n_windows
    else:
        return segment_by_num_locs_per_window(locs_nm, n_locs_per_window)


def segment_by_num_locs_per_window(locs_nm, n_locs_per_window, return_n_segments=False):
    locs_nm[:, -1] = 0
    n_locs_per_window = int(n_locs_per_window)
    n_locs = len(locs_nm[:, 0])
    n_windows = int(np.ceil(n_locs/n_locs_per_window))
    for i in range(n_windows-1):
        locs_nm[(i+1)*n_locs_per_window:, -1] += 1
    if return_n_segments:
        return locs_nm, n_windows
    else:
        return locs_nm


def segment_by_frame_windows(locs_nm, n_frames_per_seg, return_n_segments=False):
    n_frames = int(np.max(locs_nm[:, -1]) - np.min(locs_nm[:, -1]))
    n_segments = int(np.ceil(n_frames/n_frames_per_seg))
    locs_nm[:, -1] = np.floor((locs_nm[:, -1] - np.min(locs_nm[:, -1]))/n_frames_per_seg)
    if return_n_segments:
        return locs_nm, n_segments
    else:
        return locs_nm
      
if segmentation_method == "segment by number of time windows":
  localizations, n_segments = segment_by_num_windows(localizations, segmentation_parameter, True)
elif segmentation_method == "segments by number of locs per window":
  localizations, n_segments = segment_by_num_locs_per_window(localizations, segmentation_parameter, True)
elif segmentation_method == "segment by number of frames per window":
  localizations, n_segments = segment_by_frame_windows(localizations, segmentation_parameter, True)
else:
  print("Error: no such segmentation method")
#print(f"{np.min(localizations[:, -1])}, {np.max(localizations[:, -1])}")
print(f"segmentation into {n_segments} segments done")

segmentation into 300 segments done


## Step 3: Run the algorithm 

The next few lines show the heart of the comet project: the drift estimation algortihm. There is a 2D- and 3D-version of the algortihm for 2D- and 3D-datasets respectively. Run the next block of code to define the functions and the block of code after to estimate the drift of your segmented localizations. There you will find a variable called max_drift_nm with is 300 nm by default. If you're able to estimate the maximum drift in your dataset feel free to adjust the parameter, but be aware, that it's directly proportional to the calculation time of the algortihm. 

In [ ]:
from numba import cuda
import math 
from scipy.optimize import minimize
import warnings
import time

def pair_indices_lex_floor(coordinates, distance):
    for i in range(len(coordinates[0])):
        coordinates[:, i] -= np.min(coordinates[:, i])
    coordinates = np.array(np.floor(coordinates / distance), dtype=int)
    coordinates = np.array(list(map(tuple, coordinates)))

    sort_indices = np.lexsort(coordinates.T)

    # get the unique tuples and their counts
    unique_tuples, counts = np.unique(coordinates[sort_indices], axis=0, return_counts=True)

    # get the indices of the similar tuples
    similar_indices = np.split(sort_indices, np.cumsum(counts[:-1]))

    idx_i = []
    idx_j = []
    for i in range(len(similar_indices)):
        if len(similar_indices[i]) > 1:
            idc = similar_indices[i]
            for j in range(len(idc)-1):
                tmp_n_entries = (len(idc) - j) - 1
                idx_i.append(np.repeat(np.int32(idc[j]), tmp_n_entries))
                idx_j.append(idc[np.arange(tmp_n_entries) + (j+1)])
    return idx_i, idx_j

@cuda.jit
def cost_function_full_3d_chunked(d_locs_time, start_idx, chunk_size, d_idx_i, d_idx_j, d_sigma, d_sigma_factor, d_val, d_val_sum, d_deri=np.array([[]]),
                          d_locs_coords=np.array([[]]), mu=np.array([[]])):
    # Thread id in a 1D block
    tx = cuda.threadIdx.x
    # Block id in a 1D grid
    ty = cuda.blockIdx.x
    # Block width, i.e. number of threads per block
    bw = cuda.blockDim.x
    # Compute flattened index inside the array
    pos = tx + ty * bw
    if pos < chunk_size:
        d_val[pos] = (math.exp(1)**(-(((d_locs_coords[d_idx_i[pos+start_idx], 0] - mu[d_locs_time[d_idx_i[pos+start_idx]], 0]) -
                                       (d_locs_coords[d_idx_j[pos+start_idx], 0] - mu[d_locs_time[d_idx_j[pos+start_idx]], 0]))**2 +
                                      ((d_locs_coords[d_idx_i[pos+start_idx], 1] - mu[d_locs_time[d_idx_i[pos+start_idx]], 1]) -
                                       (d_locs_coords[d_idx_j[pos+start_idx], 1] - mu[d_locs_time[d_idx_j[pos+start_idx]], 1]))**2 +
                                      ((d_locs_coords[d_idx_i[pos+start_idx], 2] - mu[d_locs_time[d_idx_i[pos+start_idx]], 2]) -
                                       (d_locs_coords[d_idx_j[pos+start_idx], 2] - mu[d_locs_time[d_idx_j[pos+start_idx]], 2]))**2) /
                                    ((d_sigma*d_sigma_factor) ** 2)))
        cuda.atomic.add(d_deri, (d_locs_time[d_idx_i[pos+start_idx]], 0), d_val[pos] * 2.*(d_locs_coords[d_idx_i[pos+start_idx], 0] -
                                                                                 d_locs_coords[d_idx_j[pos+start_idx], 0] +
                                                                                 mu[d_locs_time[d_idx_j[pos+start_idx]], 0] -
                                                                                 mu[d_locs_time[d_idx_i[pos+start_idx]], 0]) /
                        (d_sigma*d_sigma_factor) ** 2)
        cuda.atomic.add(d_deri, (d_locs_time[d_idx_i[pos+start_idx]], 1), d_val[pos] * 2. * (
                    d_locs_coords[d_idx_i[pos+start_idx], 1] - d_locs_coords[d_idx_j[pos+start_idx], 1] + mu[d_locs_time[d_idx_j[pos+start_idx]], 1] -
                    mu[d_locs_time[d_idx_i[pos+start_idx]], 1]) / (d_sigma * d_sigma_factor) ** 2)
        cuda.atomic.add(d_deri, (d_locs_time[d_idx_i[pos+start_idx]], 2), d_val[pos] * 2. * (
                    d_locs_coords[d_idx_i[pos+start_idx], 2] - d_locs_coords[d_idx_j[pos+start_idx], 2] + mu[d_locs_time[d_idx_j[pos+start_idx]], 2] -
                    mu[d_locs_time[d_idx_i[pos+start_idx]], 2]) / (d_sigma * d_sigma_factor) ** 2)
        cuda.atomic.add(d_val_sum, 0, d_val[pos])
        d_val[pos] = 0


def cuda_wrapper_chunked_3d(mu, d_locs_coords, d_locs_time, d_idx_i, d_idx_j, d_sigma, d_sigma_factor, d_val, d_deri,
                 chunk_size):
    val = 0
    d_val_sum = cuda.to_device(np.zeros(1, dtype=np.float64))
    #mu = np.append([0, 0, 0], np.asarray(mu))
    mu = np.asarray(mu.reshape(int(mu.size / 3), 3), dtype=np.float64)
    n_chunks = int(np.ceil(d_idx_i.size/chunk_size))
    for i in range(n_chunks-1):
        threadsperblock = 256
        idc = np.arange(chunk_size) + i*chunk_size
        blockspergrid = (chunk_size + (threadsperblock - 1)) // threadsperblock
        cost_function_full_3d_chunked[blockspergrid, threadsperblock](d_locs_time, idc[0], chunk_size, d_idx_i, d_idx_j, d_sigma,
                                                              d_sigma_factor, d_val, d_val_sum, d_deri, d_locs_coords, mu)
        val += d_val_sum.copy_to_host()

    n_remaining = d_idx_i.size - (n_chunks - 1) * chunk_size
    idc = np.arange(n_remaining) + (n_chunks - 1) * chunk_size
    threadsperblock = 256
    blockspergrid = (n_remaining + (threadsperblock - 1)) // threadsperblock
    cost_function_full_3d_chunked[blockspergrid, threadsperblock](d_locs_time, idc[0], n_remaining, d_idx_i, d_idx_j, d_sigma,
                                                          d_sigma_factor, d_val, d_val_sum, d_deri, d_locs_coords, mu)
    val += d_val_sum.copy_to_host()[:n_remaining]
    deri = d_deri.copy_to_host()
    d_deri[:] = 0
    return -np.nansum(val), -deri.flatten()


def optimize_3d_chunked(n_segments, locs_nm, sigma_nm=30, drift_max_nm=300, sigma_factor=1, threshold_estimator_nm=5,
                display_steps=False):
    drift_estimate = np.zeros(3 * n_segments)
    bounds = []
    for i in range(n_segments):
        bounds.append(((i + 1) * -drift_max_nm, (i + 1) * drift_max_nm))
        bounds.append(((i + 1) * -drift_max_nm, (i + 1) * drift_max_nm))
        bounds.append(((i + 1) * -drift_max_nm, (i + 1) * drift_max_nm))
    bounds = np.asarray(bounds)

    idx_i, idx_j = pair_indices_lex_floor(locs_nm[:, :3].copy(), drift_max_nm)

    d_locs_coords = cuda.to_device(np.asarray(locs_nm[:, :3], dtype=np.float32).copy())
    d_locs_time = cuda.to_device(np.asarray(locs_nm[:, 3], dtype=np.int32).copy())
    idx_i = np.asarray(np.concatenate(idx_i).ravel(), dtype=np.int32)
    idx_j = np.asarray(np.concatenate(idx_j).ravel(), dtype=np.int32)
    if len(idx_i) * 32 / 8 > 1000000000:  # array size in the GB range -> use mapped memory
        print("huge number of pairs: using mapped memory")
        d_idx_i = cuda.mapped_array_like(idx_i, wc=True)
        d_idx_j = cuda.mapped_array_like(idx_j, wc=True)
        d_idx_i[:] = idx_i
        d_idx_j[:] = idx_j
    else:
        d_idx_i = cuda.to_device(idx_i)
        d_idx_j = cuda.to_device(idx_j)
    #del idx_i
    #del idx_j

    d_sigma = np.float64(sigma_nm)
    chunk_size = int(1E7)
    d_val = cuda.to_device(np.zeros(int(chunk_size)))
    deri = np.zeros((n_segments, 3), dtype=np.float64)
    d_deri = cuda.to_device(deri)
    print("preparation done: now starting the algorithm")
    optimization_done = 0
    fails = 0
    drift_estimate_gradient = np.inf
    while optimization_done == 0:
        d_sigma_factor = np.float64(sigma_factor)
        result = minimize(cuda_wrapper_chunked_3d, drift_estimate, method='L-BFGS-B', options={'disp': display_steps},
                          bounds=bounds, jac=True,
                          args=(d_locs_coords, d_locs_time, d_idx_i, d_idx_j, d_sigma, d_sigma_factor, d_val,
                                d_deri, chunk_size))
        if np.mean((result['x']-drift_estimate)**2) > drift_estimate_gradient and result['success']:
            optimization_done = 1
        elif result['success']:
            sigma_factor = sigma_factor / 1.5
            drift_estimate = result['x']
            drift_estimate_gradient = np.mean((result['x']-drift_estimate)**2)
        else:
            fails += 1
            if fails > 2:
                print("restarting with bigger sigma_factor")
                sigma_factor = sigma_factor * 2
            if fails > 5:
                raise RuntimeError('L-BFGS-B Optimization failed')

    return drift_estimate

@cuda.jit
def cost_function_full_2d_chunked(d_locs_time, start_idx, chunk_size, d_idx_i, d_idx_j, d_sigma, d_sigma_factor, d_val,
                                  d_val_sum, d_deri=np.array([[]]),
                                  d_locs_coords=np.array([[]]), mu=np.array([[]])):
    # Thread id in a 1D block
    tx = cuda.threadIdx.x
    # Block id in a 1D grid
    ty = cuda.blockIdx.x
    # Block width, i.e. number of threads per block
    bw = cuda.blockDim.x
    # Compute flattened index inside the array
    pos = tx + ty * bw
    if pos < chunk_size:
        d_val[pos] = (math.exp(1) ** (
                    -(((d_locs_coords[d_idx_i[pos + start_idx], 0] - mu[d_locs_time[d_idx_i[pos + start_idx]], 0]) -
                       (d_locs_coords[d_idx_j[pos + start_idx], 0] - mu[
                           d_locs_time[d_idx_j[pos + start_idx]], 0])) ** 2 +
                      ((d_locs_coords[d_idx_i[pos + start_idx], 1] - mu[d_locs_time[d_idx_i[pos + start_idx]], 1]) -
                       (d_locs_coords[d_idx_j[pos + start_idx], 1] - mu[
                           d_locs_time[d_idx_j[pos + start_idx]], 1])) ** 2) /
                    ((d_sigma * d_sigma_factor) ** 2)))
        cuda.atomic.add(d_deri, (d_locs_time[d_idx_i[pos + start_idx]], 0),
                        d_val[pos] * 2. * (d_locs_coords[d_idx_i[pos + start_idx], 0] - d_locs_coords[
                            d_idx_j[pos + start_idx], 0] +
                                           mu[d_locs_time[d_idx_j[pos + start_idx]], 0] - mu[
                                               d_locs_time[d_idx_i[pos + start_idx]], 0])
                        / (d_sigma * d_sigma_factor) ** 2)
        cuda.atomic.add(d_deri, (d_locs_time[d_idx_i[pos + start_idx]], 1), d_val[pos] * 2. * (
                d_locs_coords[d_idx_i[pos + start_idx], 1] - d_locs_coords[d_idx_j[pos + start_idx], 1] + mu[
            d_locs_time[d_idx_j[pos + start_idx]], 1] -
                mu[d_locs_time[d_idx_i[pos + start_idx]], 1]) / (d_sigma * d_sigma_factor) ** 2)
        cuda.atomic.add(d_val_sum, 0, d_val[pos])
        d_val[pos] = 0


def cuda_wrapper_chunked(mu, d_locs_coords, d_locs_time, d_idx_i, d_idx_j, d_sigma, d_sigma_factor, d_val, d_deri,
                         chunk_size):
    #t = time.time()
    val = 0
    d_val_sum = cuda.to_device(np.zeros(1, dtype=np.float64))
    #mu = np.append([0, 0], np.asarray(mu))
    mu = np.asarray(mu.reshape(int(mu.size / 2), 2), dtype=np.float64)
    n_chunks = int(np.ceil(d_idx_i.size / chunk_size))
    #print(f"Preparing GPU calculation in {time.time() - t}s, number of chunks: {n_chunks}")

    for i in range(n_chunks - 1):
        #t = time.time()
        threadsperblock = 256
        idc = np.arange(chunk_size) + i * chunk_size
        blockspergrid = (d_idx_j.size + (threadsperblock - 1)) // threadsperblock
        cost_function_full_2d_chunked[blockspergrid, threadsperblock](d_locs_time, idc[0], chunk_size, d_idx_i, d_idx_j,
                                                                      d_sigma,
                                                                      d_sigma_factor, d_val, d_val_sum,
                                                                      d_deri, d_locs_coords, mu)
        #print(f"gpu calculation step {i+1}/{n_chunks} done in {time.time() - t}")
        val += d_val_sum.copy_to_host()
    n_remaining = d_idx_i.size - (n_chunks - 1) * chunk_size
    idc = np.arange(n_remaining) + (n_chunks - 1) * chunk_size
    threadsperblock = 256
    blockspergrid = (n_remaining + (threadsperblock - 1)) // threadsperblock
    cost_function_full_2d_chunked[blockspergrid, threadsperblock](d_locs_time, idc[0], n_remaining, d_idx_i, d_idx_j,
                                                                  d_sigma, d_sigma_factor, d_val, d_val_sum, d_deri,
                                                                  d_locs_coords, mu)
    #t = time.time()
    val += d_val_sum.copy_to_host()[:n_remaining]
    deri = d_deri.copy_to_host()
    #print(f"time to copy derivatives to host {time.time() - t}")
    d_deri[:] = 0
    return -np.nansum(val), -deri.flatten()


def optimize_2d_chunked(n_segments, locs_nm, sigma_nm=30, drift_max_nm=300, sigma_factor=1, threshold_estimator_nm=5,
                        display_steps=False):
    drift_estimate = np.zeros(2 * n_segments)
    bounds = []
    for i in range(n_segments):
        bounds.append(((i + 1) * -drift_max_nm, (i + 1) * drift_max_nm))
        bounds.append(((i + 1) * -drift_max_nm, (i + 1) * drift_max_nm))

    idx_i, idx_j = pair_indices_lex_floor(locs_nm[:, :2].copy(), drift_max_nm)

    d_locs_coords = cuda.to_device(np.asarray(locs_nm[:, :2], dtype=np.float32).copy())
    d_locs_time = cuda.to_device(np.asarray(locs_nm[:, 2].astype(int), dtype=np.int32).copy())
    idx_i = np.asarray(np.concatenate(idx_i).ravel(), dtype=np.int32)
    idx_j = np.asarray(np.concatenate(idx_j).ravel(), dtype=np.int32)
    if len(idx_i) * 32 / 8 > 1000000000:  # array size in the GB range -> use mapped memory
        print("huge number of pairs: using mapped memory")
        d_idx_i = cuda.mapped_array_like(idx_i, wc=True)
        d_idx_j = cuda.mapped_array_like(idx_j, wc=True)
        d_idx_i[:] = idx_i
        d_idx_j[:] = idx_j
    else:
        d_idx_i = cuda.to_device(idx_i)
        d_idx_j = cuda.to_device(idx_j)
    d_sigma = np.float64(sigma_nm)
    chunk_size = int(1E7)
    d_val = cuda.to_device(np.zeros(chunk_size))
    deri = np.zeros((n_segments, 2), dtype=np.float64)
    d_deri = cuda.to_device(deri)
    print("preparation done: now starting the algorithm")
    optimization_done = 0
    fails = 0
    drift_estimate_gradient = np.inf
    while optimization_done == 0:
        d_sigma_factor = np.float64(sigma_factor)
        result = minimize(cuda_wrapper_chunked, drift_estimate, method='L-BFGS-B', options={'disp': display_steps},
                          bounds=bounds, jac=True,
                          args=(d_locs_coords, d_locs_time, d_idx_i, d_idx_j, d_sigma, d_sigma_factor, d_val, d_deri,
                                chunk_size))
        if np.mean((result['x'] - drift_estimate) ** 2) > drift_estimate_gradient and result['success']:
            optimization_done = 1
        elif result['success']:
            sigma_factor = sigma_factor / 1.5
            drift_estimate = result['x']
            drift_estimate_gradient = np.mean((result['x'] - drift_estimate) ** 2)
        else:
            fails += 1
            if fails > 2:
                print("restarting with bigger sigma_factor")
                sigma_factor = sigma_factor * 2
            if fails > 5:
                raise RuntimeError('L-BFGS-B Optimization failed')
    return drift_estimate


max_drift_nm = 300#@param {type:"number"}
max_drift_nm = float(max_drift_nm)
dataset_dimension = 3#@param [2,3]{type:"raw"}
initial_gaussian_scale_nm = 30#@param {type:"number"}
t = time.time()
#warnings.filterwarnings("ignore")
if dataset_dimension == 3:
  drift = optimize_3d_chunked(n_segments, localizations,
                              sigma_nm = initial_gaussian_scale_nm,
                              drift_max_nm=max_drift_nm,
                              display_steps=False)
  drift = drift.reshape(n_segments, 3)
else:
  drift = optimize_2d_chunked(n_segments, localizations,
                              drift_max_nm=max_drift_nm,
                              sigma_nm = initial_gaussian_scale_nm,
                              display_steps=False)
  drift = drift.reshape(n_segments, 2)
print(f"algortihm done in {np.round(time.time()-t)}s")

## Step 4: Displaying and downloading the results

In principle the algorithm is done at this point, now we want to look at the results and possibly download them for later use. 
To display the result we will use the next block of code, with shows creates a plot of the resulting drift. 

In [ ]:
import matplotlib.pyplot as plt 
from scipy.interpolate import CubicSpline 
interpolate_drift = False #@param {type:"boolean"}
if not interpolate_drift:
  plt.plot(np.arange(n_segments)/n_segments*n_frames, drift[:,0])
  plt.plot(np.arange(n_segments)/n_segments*n_frames, drift[:,1])
  if dataset_dimension == 3:
    plt.plot(np.arange(n_segments)/n_segments*n_frames, drift[:,2])
  plt.xlabel("Segments")
  plt.ylabel("Drift estimate [nm]")
else: 
  dim = len(drift[0,:])
  drift_spline = np.zeros((n_frames, dim))
  for i in range(dim):
      spline_model = CubicSpline(np.arange(n_segments), drift[:, i])
      drift_spline[:, i] = spline_model(np.arange(n_frames)/n_frames*n_segments)
  plt.plot(frames, drift_spline[:, 0])
  plt.plot(frames, drift_spline[:, 1])
  if dataset_dimension ==3:
    plt.plot(frames, drift_spline[:, 2])
  plt.xlabel("Frames")
  plt.ylabel("Drift estimate [nm]")
if dataset_dimension == 2:
  plt.legend(["x est.", "y est."])
else:
  plt.legend(["x est.", "y est.", "z est."])

Last but not least the next block of code will save the resulting drift in an .csv file, which you can simply find in the directory (folder symbol in upper left corner) after you ran the next lines of code.

In [ ]:
if dataset_dim ==3:
  header = "frame,x_nm,y_nm,z_nm\n"
else:
  header = "frame,x_nm,y_nm\n"
file_contents = (header+str(drift).replace(' [', '').replace('[', '').replace('],', ',\n').replace(']', '')).encode()
with open("results.csv", 'wb+') as f:
  f.write(file_contents)

